# Simple cancer binary classification

## Imports

In [ ]:
from sklearn.datasets import load_breast_cancer

from neuralnetlib.preprocessing import StandardScaler
from neuralnetlib.activations import Sigmoid, ReLU
from neuralnetlib.models import Model
from neuralnetlib.layers import Input, Activation, Dense, BatchNormalization
from neuralnetlib.callbacks import EarlyStopping
from neuralnetlib.losses import BinaryCrossentropy
from neuralnetlib.optimizers import Adam
from neuralnetlib.metrics import accuracy_score, f1_score, recall_score, precision_score
from neuralnetlib.utils import train_test_split

## 1. Loading a dataset (in this case, Breast Cancer dataset)

In [ ]:
data = load_breast_cancer()
X, y = data.data, data.target

## 2. Preprocessing

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

## 3. Model definition

In [ ]:
input_neurons = x_train.shape[1:][0]  # Cancer dataset has 30 features
num_hidden_layers = 5  # Number of hidden layers
hidden_neurons = 100  # Number of neurons in each hidden layer
output_neurons = 1  # Binary classification-regression

model = Model()
model.add(Input(input_neurons))
model.add(Dense(hidden_neurons, weights_init='he_uniform', random_state=42))
model.add(Activation(ReLU()))

for _ in range(num_hidden_layers - 1):
    model.add(Dense(hidden_neurons, weights_init='he_uniform', random_state=42))
    model.add(BatchNormalization())
    model.add(Activation(ReLU()))

model.add(Dense(output_neurons, random_state=42))
model.add(Activation(Sigmoid()))

## 4. Model compilation

In [ ]:
model.compile(loss_function=BinaryCrossentropy(), optimizer=Adam(learning_rate=0.0001))

model.summary()

## 5. Model training

In [ ]:
early_stopping = EarlyStopping(patience=5, min_delta=0.001, restore_best_weights=True)

history = model.fit(x_train, y_train, epochs=40, batch_size=48, random_state=42, plot_decision_boundary=True,
          callbacks=[], metrics=["accuracy"])  # Here, the early stopping will stop the training if the loss does not decrease

# You could specify a different metric because loss is the default one

# In this context, accuracy_score would have stopped at epoch 10

## 6. Model evaluation

In [ ]:
loss, _ = model.evaluate(x_test, y_test)
print(f'Test loss: {loss}')

## 7. Model prediction

In [ ]:
y_pred = model.predict(x_test)

## 8. Printing some metrics

In [ ]:
accuracy = accuracy_score(y_pred, y_test)
precision = precision_score(y_pred, y_test)
recall = recall_score(y_pred, y_test)
f1 = f1_score(y_pred, y_test)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")